In [1]:
# result: address, holdings, previousTradeTimestamp, boughtTotal, holdingCurrently

import pandas as pd
import alchemyHelper
import graphqlHelper
import json

# GLOBAL VARIABLES
uniswapV2Router = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3Router = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
weiDivisible = 1000000000000000000

# getting count
with open("mainCOUNT.txt", "r") as f:
    rowNum = int(f.read())
count = rowNum + 1

#walletsList = pd.read_csv("wallets_test.csv")
walletsList = pd.read_csv("../TokenAnalysis/testDEJITARUPARUGoodWallets.csv")
walletsList = walletsList.drop_duplicates()

tokensTraded = 0
stillHoldingOrDusted = 0
sellingIncrementally = 0
accountTransferORNULL = 0
stablecoinTransfer = 0
profits = {}

df = pd.DataFrame(columns=list(['Wallet_Address','Tokens_Traded', 'Still_Holding_OR_Dusted', 'Sold_Incrementally_(NotJet)', 'Transfer_To_Other_Account_OR_NULL', 'Transfer_Stablecoin', 'Average_Profit']))

for i, row in walletsList[rowNum:].iterrows():
    bigDICT = {}
    wallet = row["Address"]
    #print(f"CURRENT WALLET: {wallet}")

    # BUY SECTION
    buyList = alchemyHelper.getBuys(wallet)
    for dict in buyList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['buys'].append(txnHash)

    # SELL SECTION
    sellList = alchemyHelper.getSells(wallet)
    for dict in sellList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['sells'].append(txnHash)

    # bigDICT[contract]
    # ITERATING BIG DICTIONARY
    for contract in bigDICT:
        # REMOVING DUPLICATES
        bigDICT[contract]["buys"] = [*set(bigDICT[contract]["buys"])]
        bigDICT[contract]["sells"] = [*set(bigDICT[contract]["sells"])]

        # CASE0 - IRREGULARITY
        if (bigDICT[contract]["buys"] == []):
            continue

        # CASE1 - still holding token or rugged/dusted
        if (bigDICT[contract]["sells"] == []):
            stillHoldingOrDusted += 1
            continue

        # CASE2 - Great buy and selling incrementally, smart trader
        buyLen = len(bigDICT[contract]["buys"])
        sellLen = len(bigDICT[contract]["sells"])
        diffLen = sellLen / float(buyLen)
        if (diffLen > 2):
            sellingIncrementally += 1

        excludeThisContract = False
        buys = 0.0
        # CHECKING BUYS GRAPHQL
        for buy in bigDICT[contract]["buys"]:
            result = graphqlHelper.getData(buy)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                buys += result

        sells = 0.0
        # CHECKING SELLS GRAPHQL
        for sell in bigDICT[contract]["sells"]:
            result = graphqlHelper.getData(sell)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                sells += result

        profit = sells - buys
        # STABLECOIN OR JUST TRANSFER
        if (profit == 0.0 or excludeThisContract):
            continue

        # transfer maybe
        if buys == 0: continue

        profit = ((sells - buys) / buys) * 100.0
        profits[contract] = profit
        tokensTraded += 1

    #CHANGE FILE NAME HERE
    #with open("testMRTWEETGoodWallets.json", "w") as file:
    #    json.dump(profits, file)

    profitsAVG = sum(profits.values())/len(profits)

    df.loc[count] = [wallet,tokensTraded, stillHoldingOrDusted, sellingIncrementally, accountTransferORNULL, stablecoinTransfer, profitsAVG]

    count += 1

df.to_csv('RESULTS_DejitaruParu_GOOD_WALLETS.csv')


In [11]:
pd.set_option('display.max_rows', None)
table = pd.read_csv("RESULTS_KARAT_GOOD_WALLETS.csv")
table.sort_values('Average_Profit', ascending=False)

,Unnamed: 0,Wallet_Address,Tokens_Traded,Still_Holding_OR_Dusted,Sold_Incrementally_(NotJet),Transfer_To_Other_Account_OR_NULL,Transfer_Stablecoin,Average_Profit
8,9,0x8621097d2f78a56b78a774a44ee757ef5fda52c1,228,251,4,36,0,156.193597
9,10,0x1dca3a5f625fb456630ebb9383c0f01830d19bf4,266,278,5,38,0,133.667962
15,16,0xf87d0baa1efcd5c433951ee28a0ac6eb9d821c9c,415,352,7,68,4,131.087047
7,8,0xbe7779ab38864cf0a60d44264e9389f9e10a9051,177,229,1,28,0,130.321359
10,11,0xf88cf0b7c83d6a5aeb01c1f8504397c24625b84e,315,316,5,42,0,127.910259
11,12,0xda36f279bca99d11de6948d8260d612a4e3993d9,322,319,5,42,0,121.907338
16,17,0x524edffa9a42769736795b33d81c58505537acec,459,386,7,68,4,118.617706
17,18,0x1185aa173888250939438bdd13eb7e1dd9aaf9d5,460,387,7,73,4,118.607842
12,13,0x0e711c94c9d8373952f4dd5bbf28bc4808877c5c,331,324,5,45,0,118.376536
14,15,0x0a3f1926e18f855cf1829b1d614a2a4606f490a3,372,342,6,58,4,108.357416


In [12]:
pd.set_option('display.max_rows', None)
table = pd.read_csv("RESULTS_MRTWEET_GOOD_WALLETS.csv")
table.sort_values('Average_Profit', ascending=False)

,Unnamed: 0,Wallet_Address,Tokens_Traded,Still_Holding_OR_Dusted,Sold_Incrementally_(NotJet),Transfer_To_Other_Account_OR_NULL,Transfer_Stablecoin,Average_Profit
1,2,0x531e33a0bf174f89434655c95605cd235a14a52a,17,16,0,16,0,664.488548
0,1,0x9c1e088cdc957b3c1d7e72c8cecc209bd0da64d4,4,4,0,0,0,281.815471
2,3,0xe3f7b0dff7b323956a19fff00471c2bca8174eaa,48,35,3,24,1,235.717989
3,4,0x1e49f85badf4d7e763c9b51154a8f0c0ec8810f0,62,43,4,85,6,175.904428
8,9,0x4b5eb9d8c27cd2d096cd3bcc46becf6c42dca05e,111,81,8,230,6,135.272346
9,10,0xd401fe7c088990544651c63a06d2e1d9682e1174,111,85,8,290,6,135.272346
4,5,0xbc8ec6306f447a9768da5f91ccf79a0cfe76f0bb,91,61,4,85,6,129.551107
10,11,0x6695f50eada6069c8494d65654e52f4c8e43b0bf,118,90,10,348,6,129.189665
5,6,0x66e9e60620ba7d0d614f225fd08315fad694c480,92,65,5,89,6,127.862244
6,7,0x81b272e9b70f310c1ce8eda6eb06ecfd2bbeb7ba,94,73,6,222,6,123.222898
